# CNN - CIFAR 10

Creates a CNN model for CIFAR10 (10 image categories such as planes or dogs with 32x32 images in RGB). Uses some of the setup from the [PyTorch CNN tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) and the hyperparmeter guidance from https://cs231n.github.io/convolutional-networks/#comp .

In [20]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Data

We use the CIFAR10 (10 image categories) dataset, fetched and transformed as per https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html .

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
next(iter(trainloader))[0].shape

torch.Size([4, 3, 32, 32])

The images are sized 32x32x3.

## Model

Using the guidance from https://cs231n.github.io/convolutional-networks/#comp we'll want:

  - 2d convolutional layers (as the kernel is sliding over the x & y axis, but not the color channel axis.
  - 3x3 conv layers with padding of 1 to maintain the input size for the next layer ($P = (F-1)/2$).
  - Max pooling 2x2 layers
  - Sensible CNN sizes to run fairly quickly (using 1/8 of VGGNet as a guideline, but with less layers)
  - Single CNN -> POOL -> RELU combinations rather than e.g. having 2 CNN layers before pooling (as in VGGNet)
  - 3 fully connected layers at the end, using 1/8 the size of VGGNet

In [35]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 3, padding=1) # 8 output channels (1/8 of VGGNet first conv layer, for example)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1) # 16 output channels (similarly 1/8 of VGGNet)
        self.fc1 = nn.Linear(8*8*16, 512) # 512 = 1/8 of VGGNet first fully connected layer
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10) # 10 output categories
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # Flatten all dimensions (except batch)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()

# Train

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.172
[1,  4000] loss: 1.851
[1,  6000] loss: 1.636
[1,  8000] loss: 1.544
[1, 10000] loss: 1.459
[1, 12000] loss: 1.386
[2,  2000] loss: 1.288
[2,  4000] loss: 1.238
[2,  6000] loss: 1.237
[2,  8000] loss: 1.193
[2, 10000] loss: 1.181
[2, 12000] loss: 1.135
Finished Training


## Test

In [37]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 61 %


Turns out this is a little bit more accurate than the PyTorch tutorial, which is unsurprising given the larger layers.